In [1]:
import requests
from pandas import json_normalize
import pandas as pd
import numpy as np
from datetime import datetime
import json

In [2]:
countries=pd.read_csv("contries.csv")
countries.shape

(197, 3)

In [3]:
Ramadan_data=pd.DataFrame(columns=['Country','Capital','ISO','Fajr','Maghrib','Duration','Date'])
Ramadan_data

,Country,Capital,ISO,Fajr,Maghrib,Duration,Date


In [5]:
#197 record 0-196
years_h=np.arange(1414,1444,1)
FMT = '%H:%M'
years_h

array([1414, 1415, 1416, 1417, 1418, 1419, 1420, 1421, 1422, 1423, 1424,
       1425, 1426, 1427, 1428, 1429, 1430, 1431, 1432, 1433, 1434, 1435,
       1436, 1437, 1438, 1439, 1440, 1441, 1442, 1443])

In [7]:
for country_record in range(197):
    for year in years_h:
        response = requests.get(f"http://api.aladhan.com/v1/hijriCalendarByCity?city={countries.Capital[country_record]}&country={countries.ISO[country_record]}&method=2&month=09&year={year}")
        res=response.json()
        df = json_normalize(res,'data')
        f=df['timings.Fajr'][0][0:5]
        m=df['timings.Maghrib'][0][0:5]
        new_row = {
                    'Country'  :countries.Country[country_record]
                    ,'Capital'  :countries.Capital[country_record]
                    ,'ISO'     :countries.ISO[country_record]
                    ,'Fajr'    :f
                    ,'Maghrib' :m
                    ,'Duration':(datetime.strptime(m, FMT) - datetime.strptime(f, FMT)).total_seconds()/3600
                    ,'Date'    :df['date.readable'][0]
                  }
        
        Ramadan_data=Ramadan_data.append(new_row, ignore_index = True)
        

In [24]:
Ramadan_data[['Day','Month','Year']]=Ramadan_data.Date.str.split(expand=True)
Ramadan_data.head()

,Country,Capital,ISO,Fajr,Maghrib,Duration,Date,Day,Month,Year
0,Afghanistan,Kabul,AF,05:32,17:33,12.016667,12 Feb 1994,12,Feb,1994
1,Afghanistan,Kabul,AF,05:40,17:22,11.700000,01 Feb 1995,01,Feb,1995
2,Afghanistan,Kabul,AF,05:45,17:12,11.450000,22 Jan 1996,22,Jan,1996
3,Afghanistan,Kabul,AF,05:47,17:01,11.233333,10 Jan 1997,10,Jan,1997
4,Afghanistan,Kabul,AF,05:45,16:53,11.133333,31 Dec 1997,31,Dec,1997


In [25]:
Ramadan_data.to_csv('Ramadan_data.csv')